# Data Generation

In this section we generate all the data for training the model, using Haar states.

In [ ]:
# @title Pauli Matrices
# @markdown In this block we create a function to get all the Pauli matrices on an array.


def pauli():
  """Get all the Pauli Matrices."""
  from numpy import zeros, array

  s = zeros([3,2,2]) + 1j*zeros([3,2,2])
  s[0] = array([[1, 0],[0, -1]])
  s[1] = array([[0, 1],[ 1, 0]])
  s[2] = array([[0, -1j],[1j, 0]])
  return s


In [ ]:
# @title Random Haar State
# @markdown in this block we define a random Haar state generation function.

def random_haar_state(dim,rank):
  """Generate a random Haar state."""
  import numpy.random as random
  from numpy import divide, diagonal, identity, trace
  import numpy.linalg as la

  A = random.normal(0,1,(dim,dim))+1j*random.normal(0,1,(dim,dim))
  q,r = la.qr(A,mode="complete")
  r  = divide(diagonal(r),abs(diagonal(r)))*identity(dim)
  rU = q@r
  B = random.normal(0,1,(dim,rank))+1j*random.normal(0,1,(dim,rank))
  B = B@B.T.conj()
  rho = (identity(dim)+rU)@B@(identity(dim)+rU.T.conj())
  return rho/trace(rho)


In [ ]:
# @title Random Haar Pure State
# @markdown In this block we define a random Haar pure state generation function.

def random_haar_pure_state(dim,n):
  """Generate a random Haar pure state."""
  import numpy.linalg as la
  from numpy import array, dot, random

  rpure = random.normal(0,1,[dim,n]) + 1j*random.normal(0,1,[dim,n])
  rpure = rpure/la.norm(rpure,axis=0)
  rhon = array([dot(rpure[:,[i]],rpure[:,[i]].conjugate().transpose())  for i in range(n)])
  # rhon = reshape(rhon,[n,4])
  return rhon


In [ ]:
# @title Mutually Unbiased Basis Projectors
# @markdown Here we define one qubit Pauli Projectors

def mubpom():
  """Returns the 1-qubit Pauli projectors."""
  from numpy import array, zeros, sqrt, transpose, conjugate

  p1 = array([1,0])
  p2 = array([0,1])
  mub = zeros([6,1,2])+1j*zeros([6,1,2])
  mub[0] = p1
  mub[1] = p2
  mub[2] = 1/sqrt(2)*(p1+p2)
  mub[3] = 1/sqrt(2)*(p1-p2)
  mub[4] = 1/sqrt(2)*(p1+1j*p2)
  mub[5] = 1/sqrt(2)*(p1-1j*p2)
  mubp = [transpose(mub[i])@conjugate(mub[i]) for i in range(6)]
  return mubp


In [ ]:
# @title Third Mutually Unbiased Basis
# @markdown in this block we obtain the third mutually unbiased basis for the states generation.

def mub3(nProj):
  from numpy import linspace, sort, array, kron

  mubsN = linspace(0,35,nProj,dtype=int)
  mub = mubpom()
  mub2 = array([kron(mub[i],mub[j])/9 for i in range(6) for j in range(6)])
  mub3 = array([ mub2[n] for n in sort(mubsN[0:nProj]) ])
  return mub3


In [ ]:
# @title Stokes from $\rho$

def stokes_from_rho(rho,A):
  from numpy import shape, array, real, trace
  l = shape(A)[0]
  return array([ real(trace(rho@A[n])) for n in range(l) ])


In [ ]:
# @title $\rho$ from Stokes

def rho_from_stokes(stokes, A, dim):
  from numpy import shape, identity, sum

  l = shape(A)[0]
  return 1/dim*(identity(dim)+sum([stokes[i]*A[i] for i in range(l)],axis=0))


In [ ]:
# @title Probability Distribution
# @markdown In this block we define a function to get the probability distribution.

def probdists(rho0, povm):
  """Get probabilities from quantum state rho0 and POVM."""
  from numpy import shape, array, real, trace, sum

  l = shape(povm)[0]
  probtrue = array([real(trace(rho0@povm[i])) for i in range(l)])
  probtrue = probtrue/sum(probtrue)
  return probtrue


In [ ]:
# @title Concurrence
# @markdown In this block we define a function to get the real concurrence.

def conc(A):
  """Get the real concurrence of an state."""
  from numpy import kron, conjugate, real, sort
  import numpy.linalg as la
  import scipy.linalg as sa

  s = pauli()
  s2 = kron(s[2],s[2])
  At = (s2@conjugate(A))@s2
  As = sa.sqrtm(A)
  R = sa.sqrtm((As@At)@As)
  eigval = real(sort(la.eig(R.astype(complex))[0])[::-1])
  return max(0,eigval[0]-(eigval[1]+eigval[2]+eigval[3]))


In [ ]:
# @title Herbasis

def herbasis(dim):
  from numpy import zeros, identity, dot, transpose, stack, concatenate

  pom1 = zeros([1,dim,dim])+1j*zeros([1,dim,dim])
  pom1[0] = identity(dim)
  arrays = [dot(transpose(pom1[0][[i]]),pom1[0][[i]]) for i in range(dim-1)]
  pom = stack(arrays,axis=0)
  her = concatenate((pom1,pom),axis=0)
  arrays = [dot(transpose(her[0][[i]]),her[0][[j]])+dot(transpose(her[0][[j]]),her[0][[i]]) for i in range(dim) for j in range(i+1,dim)]
  pom = stack(arrays,axis=0)
  her = concatenate((her,pom),axis=0)
  arrays = [-1j*dot(transpose(her[0][[i]]),her[0][[j]])+1j*dot(transpose(her[0][[j]]),her[0][[i]]) for i in range(dim) for j in range(i+1,dim)]
  pom = stack(arrays,axis=0)
  pom = concatenate((her,pom),axis=0)
  return pom


In [ ]:
# @title Pauli for Learning

def pauli_for_learning(nProj):
  from numpy import linspace, kron, array, sort

  mubsN = linspace(0,35,nProj,dtype=int)
  mub = mubpom()
  mub2 = array([kron(mub[i],mub[j])/9 for i in range(6) for j in range(6)])
  mub3 = array([ mub2[n] for n in sort(mubsN[0:nProj]) ])
  return mub3


In [ ]:
# @title Gellmann

def gellmann(Q,dim):
  from numpy import zeros, trace, sqrt

  q = zeros([dim**2,dim,dim])+1j*zeros([dim**2,dim,dim])
  for i in range(dim**2):
    v = Q[i]
    for j in range(0,i):
      v = v-trace(v@q[j])*q[j]
    q[i] = v/sqrt(trace(v@v))
  return q


In [ ]:
# @ttile GetR

def getr():
  from numpy import array, random

  n = 0
  p = array(range(36)[::-1])**2
  p = p/sum(p)
  k = random.multinomial(1,p)
  while k[n]==0:
    n+=1
  return n,0


In [ ]:
# @title Data generation function
# @markdown This blocks defines a function which generates data for the number of states given.

def data_gen(quantity, nProj, dim, ll):
  """Generate the data to train a network."""
  from numpy import cbrt, random, arange, concatenate, sqrt, array, kron, ones, zeros, expand_dims

  Q = herbasis(dim)
  G_all = gellmann(Q,dim)*sqrt(dim)
  G = G_all[1::]

  mub = mubpom()
  mub2 = array([kron(mub[i],mub[j])/9 for i in range(6) for j in range(6) ])

  srmPOMs = array([ mub2 for k in range(quantity) ])
  pomS = ones((quantity,36),dtype=int)


  for n in range(quantity):
    myList=list(range(nProj))
    random.shuffle(myList)
    # ~ k = 0
    k = getr()[ll]
    for i in myList[0:k]:
      srmPOMs[n][i] = zeros([4,4],complex)
      pomS[n][i] = 0

  stokesFromPOMs = array([ array([ stokes_from_rho(srmPOMs[n][m],G_all) for m in range(nProj) ]) for n in range(quantity) ])

  radius_list=cbrt(cbrt(cbrt(random.rand(int(quantity/5)))))
  rhon = random_haar_pure_state(4,int(quantity/5))
  stokes_list = array([ radius_list[n]*stokes_from_rho(rhon[n], G) for n in range(int(quantity/5)) ])
  rhoList1 = array([ rho_from_stokes(stokes_list[n], G, dim) for n in range(int(quantity/5)) ])

  rhoR1 = array([random_haar_state(4,1) for k in range(int(quantity/5))])
  rhoR2 = array([random_haar_state(4,2) for k in range(int(quantity/5))])
  rhoR3 = array([random_haar_state(4,3) for k in range(int(quantity/5))])
  rhoR4 = array([random_haar_state(4,4) for k in range(int(quantity/5))])
  rho_list = concatenate((rhoList1, rhoR1,rhoR2, rhoR3, rhoR4))

  indxs = random.shuffle(arange(quantity))
  rho_list = rho_list[indxs][0]

  # ~ concurrence
  y_train = array([conc(rho_list[n]) for n in range(int(4*quantity/5))])
  y_val = array([conc(rho_list[n]) for n in range(int(4*quantity/5), quantity)])

  # ~ probabilities
  probListSRM = array([ probdists(rho_list[n],srmPOMs[n]) for n in range(quantity)])


  x_train = array([probdists(rho_list[n],srmPOMs[n]) for n in range(int(4*quantity/5))])
  x_val = array([probdists(rho_list[n],srmPOMs[n]) for n in range(int(4*quantity/5),quantity)])

  for n in range(int(quantity*4/5)):
    for i in range(nProj):
      x_train[n,i*17:i*17+16] = stokesFromPOMs[n][i]
      x_train[n,i*17+16] = probListSRM[n,i]

  for n in range(int(quantity/5)):
    for i in range(nProj):
      x_val[n,i*17:i*17+16] = stokesFromPOMs[int(quantity*4/5)+n][i]
      x_val[n,i*17+16] = probListSRM[int(quantity*4/5)+n,i]

  return x_train,x_val,y_train,y_val


# Measurement Independant DNN

In this section we define the deep neural network for the measurement independent case.

In [ ]:
def measurement_independent_dnn():
  """Return a new model without training for the measurement independent case."""
  import keras

  model = keras.Sequential()

  model.add(keras.layers.Conv1D(100,
    kernel_size=17,
    strides=17,
    input_shape=(17*36,1),
    activation="relu",
    kernel_initializer=keras.initializers.glorot_normal(seed=42)
  ))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(120, activation="relu", kernel_initializer=keras.initializers.glorot_normal(seed=42)))
  model.add(keras.layers.Dense(80, activation="relu", kernel_initializer=keras.initializers.glorot_normal(seed=42)))
  model.add(keras.layers.Dense(70, activation="relu", kernel_initializer=keras.initializers.glorot_normal(seed=42)))
  model.add(keras.layers.Dense(60, activation="relu", kernel_initializer=keras.initializers.glorot_normal(seed=42)))
  model.add(keras.layers.Dense(50, activation="relu", kernel_initializer=keras.initializers.glorot_normal(seed=42)))
  model.add(keras.layers.Dense(40, activation="relu", kernel_initializer=keras.initializers.glorot_normal(seed=42)))

  model.add(keras.layers.Dense(1, activation="sigmoid", kernel_initializer=keras.initializers.glorot_normal(seed=42)))

  model.compile(loss="mse", optimizer="Nadam", metrics=["mean_absolute_error"])

  return model


In [ ]:
# @title Model Training
# @markdown In this section we define the training strategy for the DNN.

def train_measurement_independent_dnn():
  from keras.callbacks import ModelCheckpoint
  from numpy import savetxt

  nProj = 36
  dim = 4
  noStates = 10000000

  x_train,x_val,y_train,y_val = data_gen(200000,nProj,dim,1)

  batch_size=500
  epochs=100

  bestModel = measurement_independent_dnn()
  bestMAE = bestModel.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=0,
    validation_data=(x_val, y_val)
  )
  bestModel.save("bestModelConcHaar.keras")

  for n in range(10):
    print(n)
    currentModel = measurement_independent_dnn()
    currentMAE = bestModel.fit(
      x_train, y_train,
      batch_size=batch_size,
      epochs=epochs,
      verbose=0,
      validation_data=(x_val, y_val)
    )
    if currentMAE.history["val_mean_absolute_error"][-1]<bestMAE.history["val_mean_absolute_error"][-1]:
      bestMAE=currentMAE
      bestModel=currentModel
      bestModel.save("bestModelConcHaar.keras")

    print(bestMAE.history["val_mean_absolute_error"][-1])

  x_train,x_val,y_train,y_val = data_gen(noStates,nProj,dim,0)

  filepath="bestModelConcHaar.keras"

  checkpoint = ModelCheckpoint(filepath, monitor="val_mean_absolute_error", verbose=0, save_best_only=True, mode="min")
  callbacks_list = [checkpoint]

  history = bestModel.fit(x_train, y_train,
    batch_size=100,
    epochs=2000,
    verbose=1,
    validation_data=(x_val, y_val),
    callbacks = callbacks_list
  )

  bestModel.save("bestModelConcHaar.keras")
  mse = history.history["mean_absolute_error"]
  val_mse = history.history["val_mean_absolute_error"]
  savetxt("trainingErrorsConc.txt", [mse,val_mse], fmt="%s")


In [ ]:
train_measurement_independent_dnn()
